In [ ]:
import subprocess, sys, os

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib widget

In [ ]:
sim_name_fault_emt = "EMT_Kundur2Areas_SS"
sim_name_s1 = "Cosim_Kundur2Areas_SS_S1_zoh_"
sim_name_s2 = "Cosim_Kundur2Areas_SS_S2_zoh_"

varname_dpsim = 'N8.V'
nominal_voltage = 230000

timestep_monolithic = 1e-6

H_v = [2e-6, 5e-6, 1e-5, 2.2e-5, 4.6e-5, 0.0001]
H_v_labels = {'2e-06': '2', '5e-06': '5', '1e-05': '10', '2.2e-05': '22', '4.6e-05': '46', '0.0001': '100'}

dpsim_result_file_fault = 'logs/' + sim_name_fault_emt + '/' + sim_name_fault_emt + '.csv'
ts_dpsim_fault_emt = read_timeseries_csv(dpsim_result_file_fault)

#plot parameters
width = 6
height = 4

# fig, axes = plt.subplots(figsize=(width, height))

rmse_emt_v = []
rmse_emt_percent_v = []

rmse_dp_v = []
rmse_dp_percent_v = []

dpsim_emt_values = ts_dpsim_fault_emt[varname_dpsim+"_0"].values / nominal_voltage * np.sqrt(3/2)

i = 0

for H in H_v:
    print(i)
    pref = str(H)

    dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1 + pref + '/' + sim_name_s1 + pref + '.csv'
    ts_dpsim_fault_s1_emt = read_timeseries_csv(dpsim_result_file_fault_s1)

    # dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1_dp + pref + '/' + sim_name_s1_dp + pref + '.csv'
    # results_s1 = read_timeseries_csv(dpsim_result_file_fault_s1)
    # ts_dpsim_fault_s1_dp = { k:v.frequency_shift(freq=60) for (k, v) in results_s1.items() }
    
    dpsim_emt_values_s1 = ts_dpsim_fault_s1_emt[varname_dpsim+"_0"].values / nominal_voltage * np.sqrt(3/2)
    # dpsim_dp_values_s1 = ts_dpsim_fault_s1_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)

    if len(dpsim_emt_values) != len(dpsim_emt_values_s1):
        print(str(dpsim_emt_values[-1]) + " differs from " + str(dpsim_emt_values_s1[-1]))
    
    rmse_emt_t = np.abs(dpsim_emt_values - dpsim_emt_values_s1)
    rmse_emt = np.sqrt(((dpsim_emt_values - dpsim_emt_values_s1) ** 2).mean())
    rmse_emt_percent = rmse_emt/1.0*100
    rmse_emt_v.append(rmse_emt)
    rmse_emt_percent_v.append(rmse_emt_percent)

    # rmse_dp = np.sqrt(((dpsim_dp_values - dpsim_dp_values_s1) ** 2).mean())
    # rmse_dp_percent = rmse_dp/1.0*100
    # rmse_dp_v.append(rmse_dp)
    # rmse_dp_percent_v.append(rmse_dp_percent)

    # axes.plot(ts_dpsim_fault_s1_emt[varname_dpsim+"_0"].time, rmse_emt_t)

    i += 1

fig, axes = plt.subplots(2, 1, figsize=(width, height))
axes[0].plot(np.log10(H_v), np.log10(rmse_emt_v), 'o-')
axes[0].grid()
axes[0].set_xlabel("H (ms)")
# axes[0].set_xticks(H_v, labels=H_v_labels)
axes[0].set_ylabel("RMSE - EMT")
# axes[1].plot(H_v, rmse_dp_v, 'o-r')
# axes[1].grid()
# axes[1].set_xlabel("H (ms)")
# axes[1].set_xticks(H_v, labels=H_v_labels)
# axes[1].set_ylabel("RMSE - DP")

In [ ]:
sim_name_fault_emt = "EMT_Kundur2Areas_Fault"
sim_name_s1 = "Cosim_Kundur2Areas_Fault_S1_zoh_"
sim_name_s2 = "Cosim_Kundur2Areas_Fault_S2_zoh_"

varname_dpsim = 'N8.V'
nominal_voltage = 230000

timestep_monolithic = 1e-6

H_v = [2e-6, 5e-6, 1e-5, 2.2e-5, 4.6e-5, 0.0001]
H_v_labels = {'2e-06': '2', '5e-06': '5', '1e-05': '10', '2.2e-05': '22', '4.6e-05': '46', '0.0001': '100'}

dpsim_result_file_fault = 'logs/' + sim_name_fault_emt + '/' + sim_name_fault_emt + '.csv'
ts_dpsim_fault_emt = read_timeseries_csv(dpsim_result_file_fault)

#plot parameters
width = 6
height = 4

rmse_emt_v = []
rmse_emt_percent_v = []

rmse_dp_v = []
rmse_dp_percent_v = []

dpsim_emt_values = ts_dpsim_fault_emt[varname_dpsim+"_0"].values / nominal_voltage * np.sqrt(3/2)

i = 0

for H in H_v:
    print(i)
    pref = str(H)

    dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1 + pref + '/' + sim_name_s1 + pref + '.csv'
    ts_dpsim_fault_s1_emt = read_timeseries_csv(dpsim_result_file_fault_s1)

    # dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1_dp + pref + '/' + sim_name_s1_dp + pref + '.csv'
    # results_s1 = read_timeseries_csv(dpsim_result_file_fault_s1)
    # ts_dpsim_fault_s1_dp = { k:v.frequency_shift(freq=60) for (k, v) in results_s1.items() }
    
    dpsim_emt_values_s1 = ts_dpsim_fault_s1_emt[varname_dpsim+"_0"].values / nominal_voltage * np.sqrt(3/2)
    # dpsim_dp_values_s1 = ts_dpsim_fault_s1_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)

    if len(dpsim_emt_values) != len(dpsim_emt_values_s1):
        print(str(dpsim_emt_values[-1]) + " differs from " + str(dpsim_emt_values_s1[-1]))
    
    rmse_emt = np.sqrt(((dpsim_emt_values - dpsim_emt_values_s1) ** 2).mean())
    rmse_emt_percent = rmse_emt/1.0*100
    rmse_emt_v.append(rmse_emt)
    rmse_emt_percent_v.append(rmse_emt_percent)

    # rmse_dp = np.sqrt(((dpsim_dp_values - dpsim_dp_values_s1) ** 2).mean())
    # rmse_dp_percent = rmse_dp/1.0*100
    # rmse_dp_v.append(rmse_dp)
    # rmse_dp_percent_v.append(rmse_dp_percent)

    i += 1

fig, axes = plt.subplots(2, 1, figsize=(width, height))
axes[0].plot(np.log10(H_v), np.log10(rmse_emt_v), 'o-')
axes[0].grid()
axes[0].set_xlabel("H (ms)")
# axes[0].set_xticks(H_v, labels=H_v_labels)
axes[0].set_ylabel("RMSE - EMT")
# axes[1].plot(H_v, rmse_dp_v, 'o-r')
# axes[1].grid()
# axes[1].set_xlabel("H (ms)")
# axes[1].set_xticks(H_v, labels=H_v_labels)
# axes[1].set_ylabel("RMSE - DP")

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

sim_name_fault_emt = "EMT_Kundur2Areas_Fault"
sim_name_s1 = "Cosim_Kundur2Areas_Fault_S1_zoh_"
sim_name_s2 = "Cosim_Kundur2Areas_Fault_S2_zoh_"

H_v = [4e-06, 0.0001]
prefixes = ['4e-06', '0.0001']
prefix_labels = {'4e-06': '4', '0.0001': '100'}

dpsim_result_file_fault = 'logs/' + sim_name_fault_emt + '/' + sim_name_fault_emt + '.csv'
ts_dpsim_fault_emt = read_timeseries_csv(dpsim_result_file_fault)

# timestep_common = 4e-6
timestep_monolithic = 1e-6

varname_dpsim = 'N8.V'
nominal_voltage = 230000

t_begin = 0.0
t_end = 2
# begin_idx = int(t_begin/timestep_common)
# end_idx= int(t_end/timestep_common)
# time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 6
height = 4

fig, axes = plt.subplots(figsize=(width, height))

#convert dpsim voltage to magnitude value and per-unit for comparison with psat
# dpsim_emt_values = ts_dpsim_fault_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)

m = int(np.around(min(H_v)/timestep_monolithic))

time = ts_dpsim_fault_emt[varname_dpsim+"_0"].time[0:-1:m]
dpsim_emt_values = ts_dpsim_fault_emt[varname_dpsim+"_0"].values[0:-1:m] / nominal_voltage * np.sqrt(3/2)

#second axis of first plot
roi_start_second=0.095
roi_end_second=0.105
ylim = axes.get_ylim()
# roi = range(int(roi_start_second/timestep_common), int(roi_end_second/timestep_common))
# ax2 = axes.inset_axes([0.35, 0.45, .55, .55])
# x1, x2, y1, y2 = roi_start_second, roi_end_second, -0.4, 1
x1, x2, y1, y2 = roi_start_second, roi_end_second, -0.5, 1
# mark_inset(axes, ax2, loc1=2, loc2=3, fc="none", color='black')

i = 0

for pref in prefixes:    
    dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1 + pref + '/' + sim_name_s1 + pref + '.csv'
    ts_dpsim_fault_s1_emt = read_timeseries_csv(dpsim_result_file_fault_s1)
                                 
    dpsim_result_file_fault_s2 = 'logs/' + sim_name_s2 + pref + '/' + sim_name_s2 + pref + '.csv'
    ts_dpsim_fault_s2_emt = read_timeseries_csv(dpsim_result_file_fault_s2)

    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    # dpsim_emt_values_s1 = ts_dpsim_fault_s1_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
    # dpsim_emt_values_s2 = ts_dpsim_fault_s2_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
    
    dpsim_emt_values_s1 = ts_dpsim_fault_s1_emt[varname_dpsim+"_0"].values / nominal_voltage * np.sqrt(3/2)

    # plt.subplot(1, 2, 1)
    # axes.plot(time, dpsim_emt_values_s1, '--', label='EMT - Cosimulation ' + prefix_labels[pref])
    axes.plot(ts_dpsim_fault_s1_emt[varname_dpsim+"_0"].time, dpsim_emt_values_s1, '--', label='EMT - Cosimulation ' + prefix_labels[pref])
    # plt.plot(time, dpsim_emt_values_s2, '--', label='EMT - DPsim - S2')
    axes.set_ylim((-1.5,3))
    # axes.set_xlim([0.05, 0.25])
    axes.set_xlabel("time (s)")
    axes.set_ylabel(varname_dpsim + " (pu)")

    # ax2.plot(time[roi], dpsim_emt_values_s1[roi], '--')
    # ax2.set_xlim(x1, x2)
    # ax2.set_ylim(y1, y2)

    i += 1

plt.tight_layout()

axes.plot(time, dpsim_emt_values, label='EMT - Monolithic')
# ax2.plot(time[roi], dpsim_emt_values[roi])
# axes.grid()
plt.legend(loc='lower right')
# ax2.set_xticklabels([])
# ax2.set_yticklabels([])
# ax2.tick_params(top=False, bottom=False, left=False, right=False, labelleft=True, labelbottom=True)
# ax2.grid()
plt.savefig('Kundur2Areas_dyn_EMT_cosim_attributes_fault_{}_comparison.pdf'.format(varname_dpsim))


In [ ]:

# sim_name_fault_dp = "DP_Kundur2Areas_SS"
# sim_name_s1_dp = "DP_Cosim_Kundur2Areas_SS_S1_zoh_"
# sim_name_s2_dp = "DP_Cosim_Kundur2Areas_SS_S2_zoh_"

# H_v = [2e-6, 4e-6, 7e-6, 1.4e-5, 2.7e-5, 5.2e-5, 1e-4]
# prefixes = ['2e-06', '4e-06', '7e-06', '1.4e-05', '2.7e-05', '5.2e-05', '0.0001']
# prefix_labels = {'2e-06': '2', '4e-06': '4', '7e-06': '7', '1.4e-05': '14', '2.7e-05': '27', '5.2e-05': '52', '0.0001': '100'}

# dpsim_result_file_fault = 'logs/' + sim_name_fault_dp + '/' + sim_name_fault_dp + '.csv'
# results = read_timeseries_csv(dpsim_result_file_fault)
# ts_dpsim_fault_dp = { k:v.frequency_shift(freq=60) for (k, v) in results.items() }

# timestep_common = 10e-5
# varname_dpsim = 'N8.V'
# nominal_voltage = 230000

# t_begin = 0.0
# t_end = 2
# begin_idx = int(t_begin/timestep_common)
# end_idx= int(t_end/timestep_common)
# time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

# #plot parameters
# width = 6
# height = 4

# fig, axes = plt.subplots(figsize=(width, height))

# #convert dpsim voltage to magnitude value and per-unit for comparison with psat
# dpsim_dp_values = ts_dpsim_fault_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage

# #second axis of first plot
# roi_start_second=0.095
# roi_end_second=0.105
# ylim = axes.get_ylim()
# roi = range(int(roi_start_second/timestep_common), int(roi_end_second/timestep_common))
# ax2 = axes.inset_axes([0.35, 0.45, .55, .55])
# # x1, x2, y1, y2 = roi_start_second, roi_end_second, -0.4, 1
# x1, x2, y1, y2 = roi_start_second, roi_end_second, -0.5, 1
# mark_inset(axes, ax2, loc1=2, loc2=3, fc="none", color='black')

# for pref in prefixes:
#     dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1_dp + pref + '/' + sim_name_s1_dp + pref + '.csv'
#     results_s1 = read_timeseries_csv(dpsim_result_file_fault_s1)
#     ts_dpsim_fault_s1_dp = { k:v.frequency_shift(freq=60) for (k, v) in results_s1.items() }
                                 
#     dpsim_result_file_fault_s2 = 'logs/' + sim_name_s2_dp + pref + '/' + sim_name_s2_dp + pref + '.csv'
#     results_s2 = read_timeseries_csv(dpsim_result_file_fault_s2)
#     ts_dpsim_fault_s2_dp = { k:v.frequency_shift(freq=60) for (k, v) in results_s2.items() }

#     #convert dpsim voltage to magnitude value and per-unit for comparison with psat
#     dpsim_dp_values_s1 = ts_dpsim_fault_s1_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage
#     dpsim_dp_values_s2 = ts_dpsim_fault_s2_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage

#     # plt.subplot(1, 2, 1)
#     axes.plot(time, dpsim_dp_values_s1, '--', label='DP - Cosimulation ' + prefix_labels[pref])
#     # plt.plot(time, dpsim_emt_values_s2, '--', label='EMT - DPsim - S2')
#     # plt.ylim((-1.5,2.5))
#     # plt.xlim([0.05, 0.25])
#     axes.set_ylim((-1.5,3))
#     # plt.xlim([0.095, 0.105])
#     axes.set_xlabel("time (s)")
#     axes.set_ylabel(varname_dpsim + " (pu)")

#     ax2.plot(time[roi], dpsim_dp_values_s1[roi], '--')
#     ax2.set_xlim(x1, x2)
#     ax2.set_ylim(y1, y2)

# plt.tight_layout()

# axes.plot(time, dpsim_dp_values, label='DP - Monolithic')
# ax2.plot(time[roi], dpsim_dp_values[roi])
# # plt.grid()
# plt.legend(loc='lower right')
# ax2.set_xticklabels([])
# ax2.set_yticklabels([])
# ax2.tick_params(top=False, bottom=False, left=False, right=False, labelleft=True, labelbottom=True)
# ax2.grid()
# plt.savefig('Kundur2Areas_dyn_DP_cosim_attributes_fault_{}_comparison.pdf'.format(varname_dpsim))